# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

#from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.19.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

In [5]:
data_link = 'https://raw.githubusercontent.com/murphy999/Udacity-Azure-ML-Scholarship/master/nd00333-capstone/starter_file/Telco-Customer-Churn.csv'
input_data = TabularDatasetFactory.from_delimited_files(path=data_link)

In [6]:
from model_training import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(input_data)
predict_col = pd.DataFrame(y)
predict_col.head()
complete_data = pd.concat([x,predict_col],axis=1)
complete_data.head()

/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook130769/code/Users/odl_user_130769/model_training.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Partner'] = df.Partner.apply(lambda s: 1 if s == True else 0)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook130769/code/Users/odl_user_130769/model_training.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dependents'] = df.Dependents.apply(lambda s: 1 if s == True else 0)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook1307

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
0,0,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,34,1,0,1,0,1,0,...,1,0,0,0,0,0,1,1,0,0
2,0,0,0,2,1,0,1,1,0,0,...,1,0,0,0,0,0,1,0,0,1
3,0,0,0,45,0,0,1,0,1,1,...,1,0,0,1,0,0,0,1,0,0
4,0,0,0,2,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


In [7]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(complete_data,test_size=0.25)
train_data.head()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,gender_Male,InternetService_Fiber optic,InternetService_No,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Contract_One year,Contract_Two year,Churn
2276,1,1,0,44,1,1,0,0,1,1,...,0,1,0,0,0,1,0,0,0,0
5429,0,1,1,72,1,1,1,1,1,1,...,0,0,0,1,0,0,0,0,1,0
3581,0,1,1,63,1,0,1,1,1,1,...,1,0,0,0,1,0,0,0,1,0
1815,0,1,0,72,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
101,0,1,1,1,1,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [8]:
# Save training data to a csv file 
train_data.to_csv("customer_churn_traindata.csv")

In [9]:
data_store = ws.get_default_datastore()
data_store.upload(src_dir='./',target_path='./')
ftrain_data = Dataset.Tabular.from_delimited_files(path = [(data_store, './customer_churn_traindata.csv')])

Uploading an estimated of 9 files
Uploading ./automl.ipynb
Uploaded ./automl.ipynb, 1 files out of an estimated total of 9
Uploading ./customer_churn_traindata.csv
Uploaded ./customer_churn_traindata.csv, 2 files out of an estimated total of 9
Uploading ./hyperparameter_tuning.ipynb
Uploaded ./hyperparameter_tuning.ipynb, 3 files out of an estimated total of 9
Uploading ./model_training.py
Uploaded ./model_training.py, 4 files out of an estimated total of 9
Uploading ./score.py
Uploaded ./score.py, 5 files out of an estimated total of 9
Uploading ./.ipynb_checkpoints/automl-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/automl-checkpoint.ipynb, 6 files out of an estimated total of 9
Uploading ./.ipynb_checkpoints/hyperparameter_tuning-checkpoint.ipynb
Uploaded ./.ipynb_checkpoints/hyperparameter_tuning-checkpoint.ipynb, 7 files out of an estimated total of 9
Uploading ./.ipynb_checkpoints/model_training-checkpoint.py
Uploaded ./.ipynb_checkpoints/model_training-checkpoint.py, 8 files o

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 25,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=ftrain_data,
                             label_column_name="Churn",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=7,
                             **automl_settings
)

In [11]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster
Parent Run ID: AutoML_485d78ef-7111-4b37-935d-cd8dd3b26513

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing val

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_485d78ef-7111-4b37-935d-cd8dd3b26513',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2020-12-17T07:10:21.790427Z',
 'endTimeUtc': '2020-12-17T07:36:01.593237Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '7',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone-project","subscription_id":"de8aba62-c352-42be-b980-2faedf08ead8","resource_group":"aml-quickstarts-130769","workspace_name":"quick-starts-ws-130769","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":7,"y_min":null,"y_max":null,"num_clas

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.056842105263157895,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=50,
     

In [14]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

['best_fit_automl_model.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
# Register the Model
from azureml.core.model import Model
model = Model.register(workspace = ws, model_name = 'best_fit_automl_model', model_path = 'best_fit_automl_model.pkl')
print(model.name, model.id, model.version, sep='\t')

Registering model best_fit_automl_model
best_fit_automl_model	best_fit_automl_model:1	1


In [16]:
# Deploy the Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, Webservice
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
env = best_automl.get_environment()
conda_dep = CondaDependencies()

# Define the packages needed by the model and the script
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("pandas")
# Define azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")

# Adds dependencies to PythonSection of myenv
env.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script='score.py', environment=env)

deployment_config = LocalWebservice.deploy_configuration()
service = Model.deploy(ws, "customerservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

print(service.state)
print(service.scoring_uri)
print(service.swagger_uri)

Generating Docker build context.
2020/12/17 07:42:57 Downloading source code...
2020/12/17 07:42:59 Finished downloading source code
2020/12/17 07:43:00 Creating Docker network: acb_default_network, driver: 'bridge'
2020/12/17 07:43:02 Successfully set up Docker network: acb_default_network
2020/12/17 07:43:02 Setting up Docker configuration...
2020/12/17 07:43:08 Successfully set up Docker configuration
2020/12/17 07:43:08 Logging in to registry: 23e470b2b0da49cdb90679542711672a.azurecr.io
2020/12/17 07:43:09 Successfully logged into 23e470b2b0da49cdb90679542711672a.azurecr.io
2020/12/17 07:43:09 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/12/17 07:43:09 Scanning for dependencies...
2020/12/17 07:43:10 Successfully scanned dependencies
2020/12/17 07:43:10 Launching container with name: acb_step_0
Sending build context to Docker daemon  64.51kB
Step 1/15 : FROM mcr.microsoft.com/azureml/intelmpi2018.3-ubuntu16.04:2020111

Removing intermediate container dc941ff88deb
 ---> aab07238c633
Step 9/15 : ENV PATH /azureml-envs/azureml_9d32c71994bc632ea27f41c8c23c5b62/bin:$PATH
 ---> Running in f2da7eb8d7d9
Removing intermediate container f2da7eb8d7d9
 ---> 43a0fa4c1d11
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_9d32c71994bc632ea27f41c8c23c5b62
 ---> Running in 5b1d8a0176a2
Removing intermediate container 5b1d8a0176a2
 ---> cbe76815cdef
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_9d32c71994bc632ea27f41c8c23c5b62/lib:$LD_LIBRARY_PATH
 ---> Running in ebece11c44b7
Removing intermediate container ebece11c44b7
 ---> a804df029cf7
Step 12/15 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> 90354dea841f
Step 13/15 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit  /azureml-environment-setup/spark_cache.py'; fi
 ---> Running in 5a98b70d38f0
Removing intermediate container 5a98b70d38f0

ERROR:azureml._model_management._util:Error: Container has crashed. Did your init method fail?




Container Logs:
2020-12-17T07:51:29,221064671+00:00 - iot-server/run 
2020-12-17T07:51:29,221650066+00:00 - rsyslog/run 
2020-12-17T07:51:29,222881156+00:00 - gunicorn/run 
2020-12-17T07:51:29,229332503+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_9d32c71994bc632ea27f41c8c23c5b62/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9d32c71994bc632ea27f41c8c23c5b62/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9d32c71994bc632ea27f41c8c23c5b62/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9d32c71994bc632ea27f41c8c23c5b62/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9d32c71994bc632ea27f41c8c23c5b62/lib/libssl.so.1.0.0: no version information available (required by /usr/sbi

WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [ ]:
# Save Environment Details to a file
env.save_to_directory('./',overwrite=False)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

# URL for the web service
scoring_uri = '<your web service URI>'

# Set the content type
headers = {'Content-Type': 'application/json'}

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [
                "gender" :"Male",
                "SeniorCitizen" :0,
                "Dependents" :"No",
                "Partner" : "No"
                "tenure" :34,
                "PhoneService" :"Yes",
                "MultipleLines" :"No",
                "InternetService" :"DSL",
                "OnlineSecurity" :"Yes",
                "OnlineBackup" :"No",
                "DeviceProtection" :'Yes',
                "TechSupport" : "No",
                "StreamingTV" : "No",
                "StreamingMovies" : "No",
                "Contract" : "One year",
                "PaperlessBilling" : "No",
                "PaymentMethod" : "Mailed check",
                "MonthlyCharges" : 56.95,
                "TotalCharges" : 1889.5
            ],
            [
                "gender" :"Female",
                "SeniorCitizen" :0,
                "Dependents" :"No",
                "Partner" : "No"
                "tenure" :2,
                "PhoneService" :"Yes",
                "MultipleLines" :"Yes",
                "InternetService" :"Fiber optic",
                "OnlineSecurity" :"No",
                "OnlineBackup" :"No",
                "DeviceProtection" :'No',
                "TechSupport" : "No",
                "StreamingTV" : "No",
                "StreamingMovies" : "Yes",
                "Contract" : "Month-to-month",
                "PaperlessBilling" : "Yes",
                "PaymentMethod" : "Electronic check",
                "MonthlyCharges" : 85.7,
                "TotalCharges" : 169.8]
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)

print(resp.status_code)
print(resp.text)
print(resp.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
# Web Service Logs
print(svcs.get_logs())

In [ ]:
# Delete the service
service.delete()